In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Business Problem 

Your company is expanding in to new industries to diversify its portfolio. Specifically, they are interested in purchasing and operating airplanes for commercial and private enterprises, but do not know anything about the potential risks of aircraft. You are charged with determining which aircraft are the lowest risk for the company to start this new business endeavor. You must then translate your findings into actionable insights that the head of the new aviation division can use to help decide which aircraft to purchase.

## Question to Answer
Which aircraft types/models/makes are involved in the least severe, least fatal, or least frequent incidents?

In [2]:
#Loading the CSV file into a pandas DataFrame using Latin-1 encoding to avoid Unicode errors
aviation_df = pd.read_csv("Data\AviationData.csv", encoding='latin1')

# Display the first 5 rows of the DataFrame to preview the data
aviation_df.head()

C:\Users\user\AppData\Local\Temp\ipykernel_17828\1515961502.py:2: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  aviation_df = pd.read_csv("Data\AviationData.csv", encoding='latin1')


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


## Initial Data Exploration

After successfully loading the aviation dataset, we begin by exploring the structure and content of the data. This step helps us understand what kind of information we are working with, how clean the data is, and what areas may require attention before analysis.

### Dataset Overview

We first check the shape of the dataset, column names, data types, and a summary of the data using `.shape`, `.columns`, `.info()`, and `.describe()` methods.



In [3]:
# Get the number of rows and columns in the aviation dataset
aviation_df.shape

(88889, 31)

In [4]:
# View the column names in the dataset
aviation_df.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status',
       'Publication.Date'],
      dtype='object')

In [5]:
# See the data types and count of non-null values for each column
aviation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

In [6]:
aviation_df.describe()

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,82805.000000,77488.000000,76379.000000,76956.000000,82977.000000
mean,1.146585,0.647855,0.279881,0.357061,5.325440
std,0.446510,5.485960,1.544084,2.235625,27.913634
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000
max,8.000000,349.000000,161.000000,380.000000,699.000000


To gain more detailed insight into all columns both numerical and categorical  we use `describe(include="all")`. This provides a comprehensive view of how complete each column is and how diverse the entries are.

In [7]:
# Get summary statistics for all columns, including object (text) types
aviation_df.describe(include="all")

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
count,88889,88889,88889,88889,88837,88663,34382,34373,50132,52704,...,82697,16648,77488.000000,76379.000000,76956.000000,82977.000000,84397,61724,82505,75118
unique,87951,2,88863,14782,27758,219,25592,27156,10374,24870,...,26,13590,NaN,NaN,NaN,NaN,4,12,17074,2924
top,20001212X19172,Accident,CEN22LA149,1984-06-30,"ANCHORAGE, AK",United States,332739N,0112457W,NONE,Private,...,Personal,Pilot,NaN,NaN,NaN,NaN,VMC,Landing,Probable Cause,25-09-2020
freq,3,85015,2,25,434,82248,19,24,1488,240,...,49448,258,NaN,NaN,NaN,NaN,77303,15428,61754,17019
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.647855,0.279881,0.357061,5.325440,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.485960,1.544084,2.235625,27.913634,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,0.000000,0.000000,1.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,0.000000,0.000000,2.000000,NaN,NaN,NaN,NaN


## Selected Columns for Analysis

To help identify the lowest risk aircraft for commercial and private operations, we selected the following columns based on relevance to incident severity and aircraft performance:

- **Make** – The manufacturer of the aircraft .
- **Model** – The specific aircraft model.
- **Injury.Severity** – Classification of the incident's outcome.
- **Total.Fatal.Injuries** – Number of fatalities resulting from the incident.
- **Total.Serious.Injuries** – Number of people who sustained serious (but non-fatal) injuries.
- **Total.Minor.Injuries** – Number of people who sustained minor injuries.
- **Total.Uninjured** – Number of people involved who were not injured.
- **Aircraft.damage** – The extent of the damage to the aircraft .
- **Purpose.of.flight** – The purpose of the flight when the incident occurred .

These features provide a wholistic view of safety, survivability, and incident patterns across different aircraft types.

In [8]:
selected_columns =  ['Make', 'Model','Injury.Severity', 'Total.Fatal.Injuries', 'Total.Serious.Injuries','Total.Minor.Injuries', 'Total.Uninjured','Aircraft.damage', 'Purpose.of.flight']

aviation_df[selected_columns].head()

,Make,Model,Injury.Severity,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Aircraft.damage,Purpose.of.flight
0,Stinson,108-3,Fatal(2),2.0,0.0,0.0,0.0,Destroyed,Personal
1,Piper,PA24-180,Fatal(4),4.0,0.0,0.0,0.0,Destroyed,Personal
2,Cessna,172M,Fatal(3),3.0,NaN,NaN,NaN,Destroyed,Personal
3,Rockwell,112,Fatal(2),2.0,0.0,0.0,0.0,Destroyed,Personal
4,Cessna,501,Fatal(1),1.0,2.0,NaN,0.0,Destroyed,Personal


In [9]:
# Check for missing values
missing_values = aviation_df[selected_columns].isnull().sum()
print("Missing values per selected column:\n")
print(missing_values)

Missing values per selected column:

Make                         63
Model                        92
Injury.Severity            1000
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Aircraft.damage            3194
Purpose.of.flight          6192
dtype: int64


## Fill in Missing Values

 Handling Missing Values

We address missing data in our selected columns to ensure consistency in analysis:

- **Categorical Fields**:  
  `'Make'`, `'Model'`, `'Aircraft.damage'`, `'Injury.Severity'`, and `'Purpose.of.flight'`  
  → Missing values are replaced with `'unknown'` to indicate the absence of data **without making assumptions**.

- **Numeric Field**:  
  `'Total.Fatal.Injuries'`  
  → Missing values are replaced with `0` to **safely assume no fatalities** where none were reported. This aligns with our goal of identifying **low-risk aircraft**.

- **Note**:  
  The argument `inplace=True` ensures that these changes are **applied directly to the original DataFrame**, rather than creating a new one.

In [10]:
# Fill missing values in the selected columns

aviation_df.fillna({
    'Make': 'unknown',                        # Fill missing aircraft manufacturers with 'unknown'
    'Model': 'unknown',                       # Fill missing aircraft models with 'unknown'
    'Injury.Severity': 'unknown',             # Fill missing severity info with 'unknown'
    'Total.Fatal.Injuries': 0,                # Assume 0 if no fatal injuries are reported
    'Total.Serious.Injuries': 0,              # Assume 0 if no serious injuries are reported
    'Total.Minor.Injuries': 0,                # Assume 0 if no minor injuries are reported
    'Total.Uninjured': 0,                     # Assume 0 if uninjured data is missing
    'Aircraft.damage': 'unknown',             # Fill missing aircraft damage data with 'unknown'
    'Purpose.of.flight': 'unknown'            # Fill missing flight purpose info with 'unknown'
}, inplace=True)

aviation_df[selected_columns]

,Make,Model,Injury.Severity,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Aircraft.damage,Purpose.of.flight
0,Stinson,108-3,Fatal(2),2.0,0.0,0.0,0.0,Destroyed,Personal
1,Piper,PA24-180,Fatal(4),4.0,0.0,0.0,0.0,Destroyed,Personal
2,Cessna,172M,Fatal(3),3.0,0.0,0.0,0.0,Destroyed,Personal
3,Rockwell,112,Fatal(2),2.0,0.0,0.0,0.0,Destroyed,Personal
4,Cessna,501,Fatal(1),1.0,2.0,0.0,0.0,Destroyed,Personal
...,...,...,...,...,...,...,...,...,...
88884,PIPER,PA-28-151,Minor,0.0,1.0,0.0,0.0,unknown,Personal
88885,BELLANCA,7ECA,unknown,0.0,0.0,0.0,0.0,unknown,unknown
88886,AMERICAN CHAMPION AIRCRAFT,8GCBC,Non-Fatal,0.0,0.0,0.0,1.0,Substantial,Personal
88887,CESSNA,210N,unknown,0.0,0.0,0.0,0.0,unknown,Personal


In [11]:
# This line of code converts all text in the selected columns of the aviation_df to lowercase, while leaving numeric data unchanged.
aviation_df[selected_columns] = aviation_df[selected_columns].apply(lambda x: x.str.lower() if x.dtype == "object" else x)
aviation_df[selected_columns]

,Make,Model,Injury.Severity,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Aircraft.damage,Purpose.of.flight
0,stinson,108-3,fatal(2),2.0,0.0,0.0,0.0,destroyed,personal
1,piper,pa24-180,fatal(4),4.0,0.0,0.0,0.0,destroyed,personal
2,cessna,172m,fatal(3),3.0,0.0,0.0,0.0,destroyed,personal
3,rockwell,112,fatal(2),2.0,0.0,0.0,0.0,destroyed,personal
4,cessna,501,fatal(1),1.0,2.0,0.0,0.0,destroyed,personal
...,...,...,...,...,...,...,...,...,...
88884,piper,pa-28-151,minor,0.0,1.0,0.0,0.0,unknown,personal
88885,bellanca,7eca,unknown,0.0,0.0,0.0,0.0,unknown,unknown
88886,american champion aircraft,8gcbc,non-fatal,0.0,0.0,0.0,1.0,substantial,personal
88887,cessna,210n,unknown,0.0,0.0,0.0,0.0,unknown,personal


In [12]:
# Standardizing the data by converting categorical text columns to lowercase while leaving numeric data unchanged
aviation_df[selected_columns] = aviation_df[selected_columns].apply(lambda col: col.str.lower() if col.dtype == "object" else col)
aviation_df[selected_columns]

,Make,Model,Injury.Severity,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Aircraft.damage,Purpose.of.flight
0,stinson,108-3,fatal(2),2.0,0.0,0.0,0.0,destroyed,personal
1,piper,pa24-180,fatal(4),4.0,0.0,0.0,0.0,destroyed,personal
2,cessna,172m,fatal(3),3.0,0.0,0.0,0.0,destroyed,personal
3,rockwell,112,fatal(2),2.0,0.0,0.0,0.0,destroyed,personal
4,cessna,501,fatal(1),1.0,2.0,0.0,0.0,destroyed,personal
...,...,...,...,...,...,...,...,...,...
88884,piper,pa-28-151,minor,0.0,1.0,0.0,0.0,unknown,personal
88885,bellanca,7eca,unknown,0.0,0.0,0.0,0.0,unknown,unknown
88886,american champion aircraft,8gcbc,non-fatal,0.0,0.0,0.0,1.0,substantial,personal
88887,cessna,210n,unknown,0.0,0.0,0.0,0.0,unknown,personal
